# Variance Swap 
A variance swap pays to its buyer the difference between realized variance and a pre-agreed variance strike.

For a variance notional N, the payoff for a variance swap is:
    N*(σ_R^2-σ_K^2 ),
Where σ_R^2 is the realized variance of reference asset and σ_K is the volatility strike.

Using risk-neutral pricing, the price of a variance swap is:
    V= e^(-rt_p ) N*(E(σ_R^2 )-σ_K^2 )
    
Assume the underlying price follows the below SDE in risk-neutral measure:
    dS(t)/S(t) =δdt+ σ(t,ω)dW(t)
	Assume δ is constant
	δ=0 for futures market, δ=rd-rf for foreign interest rate
	The instantaneous volatility σ is stochastic
    
U=E[1/T ∫(σ(t,ω)^2 dt]
=2/T E[-ln⁡(S(T)/F)]
Where F=S_0*e^δT is the forward price.


In [2]:
import numpy as np
import yfinance as yf
from datetime import datetime
import pandas as pd

In [13]:
def VarSwap(S0, sigmaK, T, Tp, C, P, r, N):
    F = S0 * np.exp(r*T)
    C = C.loc[C['strike'] > F]
    P = P.loc[P['strike'] <= F]
    P['weight'], C['weight'] = 1/P['strike']**2, 1/C['strike']**2
    U = 2/T * (np.dot(C['weight'], C['lastPrice'])+np.dot(P['weight'], P['lastPrice']))
    V = np.exp(-r*Tp)*N*(U-sigmaK**2)
    return V

In [18]:
spx = yf.Ticker('^SPX')
opt = spx.option_chain(date='2022-12-16')
calls = opt.calls
calls = calls[['strike', 'lastPrice']]
puts = opt.puts
puts = puts[['strike', 'lastPrice']]
S0 = 4620.64
T = 1
Tp = T + 2/365
r = 0.05
undiscounted = calls['lastPrice'] * np.exp(r*T)
callPrice = pd.concat([calls['strike'], undiscounted], axis = 1)
undiscounted = puts['lastPrice'] * np.exp(r*T)
putPrice = pd.concat([puts['strike'], undiscounted], axis = 1)
N = 100
#K = callPrice['strike']
sigmaK = 0.3
#callPrice.isin(K)
F = S0 * np.exp(r*T)

In [19]:
VarSwap(S0, sigmaK, T, Tp, callPrice, putPrice, r, N)

<ipython-input-13-51e64c53cd98>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  P['weight'], C['weight'] = 1/P['strike']**2, 1/C['strike']**2


-8.25022922781099